# House prices forecasting




Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, a real estate company would like to optimize the listing prices of the houses it is trying to sell.

We want to do this by predicting the sale price of a house given its characteristics.

If we can predict the sale price in advance, we can decrease the time to sale.

In this project i will go step by step, while explaining till I get to the desired models.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# EDA 1

The  city that a property is located in makes a difference to the sale price. 


we will start by Calculating the number of missing values of the `city`. 

 - using the data in "house_sales.csv". 

 - the output will be an object `missing_city`, that contains the number of missing values in this column. 

In [71]:
import pandas as pd
import numpy as np
data = pd.read_csv("house_sales.csv")
data.replace("--",np.nan,inplace=True)
missing_city = data["city"].isnull().sum()
print(missing_city)


73


# EDA 2 

Before we fit any models, we will need to make sure the data is clean. 



We will create a cleaned version of the dataframe `clean_data`. 

 - still using the data in "house_sales.csv". 

 - for clarification purposes, based on my viewing of the data, I have decided to handle missing values and inconsistent data as shown below


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [72]:
import pandas as pd

# Load the data
data = pd.read_csv('house_sales.csv')

# Identify and replace missing values in 'city'
data.replace("--",np.nan,inplace=True)
data['city'].fillna('Unknown', inplace=True)

# Remove missing entries for 'sale_price'
data.dropna(subset=['sale_price'], inplace=True)

# Handle missing values for 'sale_date'
data['sale_date'].fillna('2023-01-01', inplace=True)

# Handle missing values for 'months_listed'
data['months_listed'] = pd.to_numeric(data['months_listed'], errors='coerce')
mean_months_listed = data['months_listed'].mean()
data['months_listed'].fillna(round(mean_months_listed, 1), inplace=True)

# Handle missing values for 'bedrooms'
data['bedrooms'] = pd.to_numeric(data['bedrooms'], errors='coerce')
mean_bedrooms = data['bedrooms'].mean()
data['bedrooms'].fillna(round(mean_bedrooms), inplace=True)

# Clean 'house_type' column
data['house_type'] = data['house_type'].replace({
    'Det.': 'Detached',
    'Semi': 'Semi-detached',
    'Terr.': 'Terraced'
})
data['house_type'].fillna(data['house_type'].mode()[0], inplace=True)

# Clean 'area' column: Remove 'sq.m.' suffix and convert to numeric
data['area'] = data['area'].str.replace('sq.m.', '', regex=False).astype(float)
mean_area = data['area'].mean()
data['area'].fillna(round(mean_area, 1), inplace=True)

# Save the cleaned data to a new DataFrame
clean_data = data.copy()

# Display the first few rows of the cleaned DataFrame
print(clean_data)
 


      house_id        city  sale_price  ... bedrooms     house_type   area
0      1217792  Silvertown       55943  ...        2  Semi-detached  107.8
1      1900913  Silvertown      384677  ...        5       Detached  498.8
2      1174927   Riverford      281707  ...        6       Detached  542.5
3      1773666  Silvertown      373251  ...        6       Detached  528.4
4      1258487  Silvertown      328885  ...        5       Detached  477.1
...        ...         ...         ...  ...      ...            ...    ...
1495   1123892   Riverford      198661  ...        5       Detached  432.2
1496   1327295   Poppleton      358304  ...        6       Detached  599.8
1497   1058161   Riverford      176612  ...        4       Detached  359.1
1498   1822037    Teasdale      197827  ...        3       Detached  253.7
1499   1679272   Poppleton      162099  ...        3       Detached  268.7

[1500 rows x 8 columns]


# EDA 3 

One would argue that the number of bedrooms is the biggest driver of house price. 


We need to confirm this by producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question.

 - data to use - 'house_sales.csv'.

 - Our output should be a data frame named `price_by_rooms` n  should include the three columns `bedrooms`, `avg_price`, `var_price`, rounded to 1dp. for consistency. 

  

In [73]:
import pandas as pd

# Load the data
data = pd.read_csv("house_sales.csv")

# Remove '--' and convert 'area' column to numeric, handling 'sq.m.' suffix
data['area'] = data['area'].str.replace('sq.m.', '').astype(float)

# Replace '--' with NaN for other columns and handle missing values as specified
data.replace("--", pd.NA, inplace=True)
data['city'].fillna("Unknown", inplace=True)
data['sale_date'].fillna("2023-01-01", inplace=True)
data['months_listed'] = pd.to_numeric(data['months_listed'], errors='coerce')
data['months_listed'].fillna(data['months_listed'].mean(), inplace=True)
data['bedrooms'] = pd.to_numeric(data['bedrooms'], errors='coerce')
data['bedrooms'].fillna(data['bedrooms'].mean(), inplace=True)
data['house_type'].fillna(data['house_type'].mode()[0], inplace=True)
data['area'].fillna(data['area'].mean(), inplace=True)

# Convert 'sale_date' to datetime
data['sale_date'] = pd.to_datetime(data['sale_date'])

# Aggregate data by number of bedrooms
price_by_rooms = data.groupby('bedrooms').agg(
    avg_price=('sale_price', 'mean'),
    var_price=('sale_price', 'var')
).reset_index()

# Round to 1 decimal place
price_by_rooms = price_by_rooms.round(1)

# Output the resulting dataframe
print(price_by_rooms)


   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Model 1

We will fit a baseline model to predict the sale price of a house.

 1. Using the data contained in “train.csv” </br></br>

 2. Using “validation.csv” we will predict new values based on our model. We will return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be our predicted values.

In [74]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the data
train_data = pd.read_csv('train.csv')
validation_data = pd.read_csv('validation.csv')

# Define features and target variable
X_train = train_data.drop(columns=['house_id', 'sale_price'])
y_train = train_data['sale_price']
X_validation = validation_data.drop(columns=['house_id'])

# Define preprocessing for categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['bedrooms', 'area', 'months_listed']),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['city', 'house_type'])
    ]
)

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions on the validation data
predictions = model.predict(X_validation)

# Create the results DataFrame
base_result = validation_data[['house_id']].copy()
base_result['price'] = predictions

# Output the result
print(base_result)


     house_id          price
0     1331375  121527.827316
1     1630115  304386.625267
2     1645745  384760.100656
3     1336775  123976.268985
4     1888274  271186.199353
..        ...            ...
295   1986255  351916.468218
296   1896276  370138.966354
297   1758223  259024.117384
298   1752010  169120.160936
299   1651404  391289.206911

[300 rows x 2 columns]


# Model 2

We will fit a comparison model to predict the sale price of a house.

 1. We will fit our model using the data contained in “train.csv” </br></br>

 2. We will use “validation.csv” to predict new values based on our model. And return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column will contain our predicted values.

In [75]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the data
train_data = pd.read_csv('train.csv')
validation_data = pd.read_csv('validation.csv')

# Define features and target variable
X_train = train_data.drop(columns=['house_id', 'sale_price'])
y_train = train_data['sale_price']
X_validation = validation_data.drop(columns=['house_id'])

# Define preprocessing for categorical and numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['bedrooms', 'area', 'months_listed']),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['city', 'house_type'])
    ]
)

# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions on the validation data
predictions = model.predict(X_validation)

# Create the results DataFrame
compare_result = validation_data[['house_id']].copy()
compare_result['price'] = predictions

# Output the result
print(compare_result)


     house_id      price
0     1331375   82508.23
1     1630115  302637.81
2     1645745  407608.66
3     1336775  109145.24
4     1888274  266806.75
..        ...        ...
295   1986255  362960.07
296   1896276  392788.69
297   1758223  265025.58
298   1752010  178806.29
299   1651404  424689.95

[300 rows x 2 columns]
